# Detect Eye Surgery

In [1]:
import cv2
import dlib
import numpy as np
from imutils import face_utils

def eye_aspect_ratio(eye):
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def eye_area_ratio(eye, face_area):
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[0] - eye[3])
    eye_area = A * B
    return eye_area / face_area

def detect_eye_plastic_surgery(face_landmarks, face_area):
    left_eye = face_landmarks[36:42]
    right_eye = face_landmarks[42:48]

    left_eye_ratio = eye_aspect_ratio(left_eye)
    right_eye_ratio = eye_aspect_ratio(right_eye)

    left_eye_area_ratio = eye_area_ratio(left_eye, face_area)
    right_eye_area_ratio = eye_area_ratio(right_eye, face_area)

    if (left_eye_ratio > 0.37 or right_eye_ratio > 0.37) or (left_eye_area_ratio > 0.05 or right_eye_area_ratio > 0.05):
        return True
    else:
        return False

image_path = '/Users/yiweihan/Desktop/example6.png'
image = cv2.imread(image_path)
if image is None:
    print("Image not found.")
    sys.exit()

gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('/Users/yiweihan/Desktop/shape_predictor_68_face_landmarks.dat')

faces = face_detector(gray_image)
if len(faces) == 0:
    print("No faces detected.")
    sys.exit()

for face in faces:
    face_landmarks = landmark_predictor(gray_image, face)
    face_landmarks = face_utils.shape_to_np(face_landmarks)
    face_area = gray_image.size

    if detect_eye_plastic_surgery(face_landmarks, face_area):
        print("The person has done eye plastic surgery.")
    else:
        print("The person has not done eye plastic surgery.")


The person has not done eye plastic surgery.


# Detect Nose Surgery

In [2]:
import cv2
import dlib
import numpy as np
from imutils import face_utils

def analyze_nose(face_landmarks, face_width, face_height):
    nose_width = np.linalg.norm(face_landmarks[31] - face_landmarks[35])
    nose_height = np.linalg.norm(face_landmarks[27] - face_landmarks[33])
    nose_to_face_width_ratio = nose_width / face_width
    nose_to_face_height_ratio = nose_height / face_height
    return nose_to_face_width_ratio, nose_to_face_height_ratio

def analyze_nose_color(face_landmarks, image):
    left_nose_point = tuple(face_landmarks[31])
    right_nose_point = tuple(face_landmarks[35])
    nose_bridge_point = tuple(face_landmarks[27])
    left_nose_point = (left_nose_point[0] - 5, left_nose_point[1])
    right_nose_point = (right_nose_point[0] + 5, right_nose_point[1])
    left_nose_color = np.average(image[left_nose_point[1] - 2:left_nose_point[1] + 3, left_nose_point[0] - 2:left_nose_point[0] + 3])
    right_nose_color = np.average(image[right_nose_point[1] - 2:right_nose_point[1] + 3, right_nose_point[0] - 2:right_nose_point[0] + 3])
    nose_bridge_color = np.average(image[nose_bridge_point[1] - 2:nose_bridge_point[1] + 3, nose_bridge_point[0] - 2:nose_bridge_point[0] + 3])
    return (left_nose_color + right_nose_color) / 2, nose_bridge_color

def analyze_nose_bridge_straightness(face_landmarks, gray_image):
    sobel_y = cv2.Sobel(gray_image, cv2.CV_64F, 0, 1, ksize=3)
    sobel_y = np.abs(sobel_y)
    sobel_y = np.uint8(sobel_y)
    nose_bridge_start = face_landmarks[27]
    nose_bridge_end = face_landmarks[33]
    line_points = np.round(np.linspace(nose_bridge_start, nose_bridge_end, num=20)).astype(int)
    edge_intensity_sum = 0
    for point in line_points:
        edge_intensity_sum += sobel_y[point[1], point[0]]
    return edge_intensity_sum / len(line_points)

image_path = '/Users/yiweihan/Desktop/example3.png'
image = cv2.imread(image_path)
if image is None:
    print("Image not found.")
    sys.exit()
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('/Users/yiweihan/Desktop/shape_predictor_68_face_landmarks.dat')

faces = face_detector(gray_image)
if len(faces) == 0:
    print("No faces detected.")
    sys.exit()

for face in faces:
    face_landmarks = landmark_predictor(gray_image, face)
    face_landmarks = face_utils.shape_to_np(face_landmarks)
    face_width = np.linalg.norm(face_landmarks[0] - face_landmarks[16])
    face_height = np.linalg.norm(face_landmarks[8] - face_landmarks[27])
    nose_to_face_width_ratio, nose_to_face_height_ratio = analyze_nose(face_landmarks, face_width, face_height)
    side_nose_color, nose_bridge_color = analyze_nose_color(face_landmarks, image)
    nose_bridge_straightness = analyze_nose_bridge_straightness(face_landmarks, gray_image)



if (nose_to_face_width_ratio < 0.18 or nose_to_face_height_ratio > 0.6) and (nose_bridge_straightness > 19) and (side_nose_color / nose_bridge_color) < 1:
    print("The person has done nose surgery.")
else:
    print("The person has not done nose surgery.")




The person has not done nose surgery.


# Detect Face Surgery

In [3]:
import cv2
import dlib
import numpy as np
from imutils import face_utils

def analyze_face_contour(face_landmarks, face_width):
    chin_width = np.linalg.norm(face_landmarks[5] - face_landmarks[11])
    chin_to_face_width_ratio = chin_width / face_width
    return chin_to_face_width_ratio

def angle_between_points(p1, p2, p3):
    a = np.linalg.norm(p3 - p2)
    b = np.linalg.norm(p1 - p2)
    c = np.linalg.norm(p1 - p3)
    angle = np.arccos((a**2 + b**2 - c**2) / (2 * a * b))
    angle = np.degrees(angle)
    return angle

def analyze_chin_shape(face_landmarks):
    chin_angle = angle_between_points(face_landmarks[5], face_landmarks[8], face_landmarks[11])
    if chin_angle >= 100 and chin_angle <= 150:
        return True
    return False



def analyze_face_contour_smoothness(face_landmarks, gray_image):
    sobel_x = cv2.Sobel(gray_image, cv2.CV_64F, 1, 0, ksize=3)
    sobel_x = np.abs(sobel_x)
    sobel_x = np.uint8(sobel_x)
    contour_points = face_landmarks[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]]
    edge_intensity_sum = 0
    for point in contour_points:
        edge_intensity_sum += sobel_x[point[1], point[0]]
    smoothness = edge_intensity_sum / len(contour_points)
    return smoothness

def analyze_jawline_angle(face_landmarks):
    left_jaw_angle = np.arctan2(face_landmarks[5][1] - face_landmarks[1][1],
                                face_landmarks[5][0] - face_landmarks[1][0])
    right_jaw_angle = np.arctan2(face_landmarks[11][1] - face_landmarks[15][1],
                                 face_landmarks[11][0] - face_landmarks[15][0])
    left_jaw_angle = np.degrees(left_jaw_angle)
    right_jaw_angle = np.degrees(right_jaw_angle)
    return left_jaw_angle, right_jaw_angle

def analyze_face_contour_symmetry(face_landmarks):
    left_contour_points = face_landmarks[[1, 2, 3, 4, 5, 6, 7, 8]]
    right_contour_points = face_landmarks[[15, 14, 13, 12, 11, 10, 9, 8]]

    right_contour_points_flipped = np.flip(right_contour_points, axis=0)
    flipped_and_reflected_right_contour_points = np.copy(right_contour_points_flipped)
    flipped_and_reflected_right_contour_points[:, 0] = -flipped_and_reflected_right_contour_points[:, 0]

    diff = np.linalg.norm(left_contour_points - flipped_and_reflected_right_contour_points, axis=1)
    average_diff = np.mean(diff)

    return average_diff


image_path = '/Users/yiweihan/Desktop/example2.png'
image = cv2.imread(image_path)
if image is None:
    print("Image not found.")
    sys.exit()
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('/Users/yiweihan/Desktop/shape_predictor_68_face_landmarks.dat')

faces = face_detector(gray_image)
if len(faces) == 0:
    print("No faces detected.")
    sys.exit()

for face in faces:
    face_landmarks = landmark_predictor(gray_image, face)
    face_landmarks = face_utils.shape_to_np(face_landmarks)

    is_v_shaped_chin = analyze_chin_shape(face_landmarks)
    face_contour_smoothness = analyze_face_contour_smoothness(face_landmarks, gray_image)
    face_width = np.linalg.norm(face_landmarks[0] - face_landmarks[16])
    chin_to_face_width_ratio = analyze_face_contour(face_landmarks, face_width)
    left_jaw_angle, right_jaw_angle = analyze_jawline_angle(face_landmarks)
    face_contour_symmetry = analyze_face_contour_symmetry(face_landmarks)
    symmetry_threshold = 450

    if (chin_to_face_width_ratio < 0.65 or left_jaw_angle > 60 or right_jaw_angle > 60) and (face_contour_smoothness < 86 and \
        face_contour_symmetry < symmetry_threshold):
        print("The person has done face contour surgery.")
    else:
        print("The person has not done face contour surgery.")


The person has not done face contour surgery.
